In [ ]:
%pip install ortools

In [5]:
from ortools.sat.python import cp_model

def optimize_budget_with_weighted_quadratic_loss_custom_units(current_allocations, income, scale=1000):
    """
    Optimize budget allocation while penalizing deviations from current allocations using a weighted quadratic loss.
    In this version, the allocations are constrained to be in fixed steps:
      - 'transport' is allocated in multiples of 10 dollars.
      - All other categories are allocated in multiples of 50 dollars.
    
    The loss for each category is:
      weight[cat] * (effective_allocation[cat] - current_allocations[cat])^2
    where effective_allocation[cat] = factor[cat] * (unit variable for cat).
    
    Arguments:
      current_allocations: dict mapping category names (e.g., 'savings', 'housing', etc.) to their current dollar allocations.
      income: total monthly take-home income (in dollars).
      scale: a constant used to determine the weight for each category.
      
    Returns:
      A tuple (result, total_weighted_quad_loss) where:
        - result is a dict with the effective (dollar) allocation for each category,
        - total_weighted_quad_loss is the sum of the weighted quadratic losses.
    """
    model = cp_model.CpModel()
    
    # Define budget categories.
    categories = ['transport', 'food', 'housing', 'insurance', 'other_needs',
                  'investment', 'savings', 'total_needs', 'total_wants']
    
    # Define the step factor for each category:
    # For transport we want increments of 10 dollars and for others increments of 50 dollars.
    factors = {}
    for cat in categories:
        if cat == 'transport':
            factors[cat] = 10
        else:
            factors[cat] = 50
    
    # We update current allocations for total_needs and total_wants based on the provided
    # breakdown. For example, we set current total_needs as the sum of transport, food, housing,
    # insurance, and other_needs. Then current_total_wants is derived to ensure total income
    # equals savings + total_needs + total_wants.
    current_total_needs = 0
    for cat in ['transport', 'food', 'housing', 'insurance', 'other_needs']:
        current_total_needs += current_allocations.get(cat, 0)
    current_total_wants = income - current_total_needs - current_allocations.get('savings', 0)
    current_allocations['total_needs'] = current_total_needs
    current_allocations['total_wants'] = current_total_wants
    
    # Compute a weight for each category based on its current allocation.
    # A lower current allocation gives a higher weight.
    weights = {}
    for cat in categories:
        current_val = current_allocations.get(cat, 0)
        weights[cat] = scale / (current_val + 1)
        # Round to an integer (with a minimum weight of 1).
        weights[cat] = max(1, int(round(weights[cat])))
    
    # Dictionaries to hold decision variables.
    unit_vars = {}   # decision variables in units
    alloc_vars = {}  # effective allocations (in dollars)
    quad_vars = {}   # quadratic loss variables (deviation squared)
    weighted_quad_terms = []  # objective terms
    
    # Create decision variables for each category.
    for cat in categories:
        factor = factors[cat]
        max_units = income // factor  # maximum units so that effective allocation <= income.
        unit_vars[cat] = model.NewIntVar(0, max_units, f"{cat}_units")
        
        # Effective dollar allocation for this category.
        alloc_vars[cat] = model.NewIntVar(0, income, f"{cat}_allocation")
        model.Add(alloc_vars[cat] == unit_vars[cat] * factor)
        
        # Compute deviation from the current allocation (in dollars).
        current_val = current_allocations.get(cat, 0)
        dev = model.NewIntVar(-income, income, f"dev_{cat}")
        model.Add(dev == alloc_vars[cat] - current_val)
        
        # Compute quadratic loss for the deviation.
        quad = model.NewIntVar(0, income**2, f"quad_{cat}")
        model.AddMultiplicationEquality(quad, [dev, dev])
        quad_vars[cat] = quad
        
        # Multiply by the weight for this category.
        weighted_term = weights[cat] * quad
        weighted_quad_terms.append(weighted_term)
    
    # --- Budget Constraints ---
    # For example, we enforce that the sum of the effective allocations for the needs categories equals total_needs.
    model.Add(sum(alloc_vars[cat] for cat in ['transport', 'food', 'housing', 'insurance', 'other_needs'])
              == alloc_vars['total_needs'])
    
    # Total allocation constraint: total_needs + total_wants + savings must equal the total income.
    model.Add(sum(alloc_vars[cat] for cat in ['total_needs', 'total_wants', 'savings']) == income)
    
    # Hard constraints (adjust these bounds as needed):
    model.Add(alloc_vars['savings'] >= int(0.2 * income))
    model.Add(alloc_vars['total_needs'] <= int(0.5 * income))
    model.Add(alloc_vars['total_wants'] <= int(0.3 * income))
    
    # --- Objective: minimize total weighted quadratic loss ---
    model.Minimize(sum(weighted_quad_terms))
    
    # Solve the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        # Extract effective allocations (dollar amounts).
        result = {cat: solver.Value(alloc_vars[cat]) for cat in categories}
        total_weighted_quad_loss = sum(solver.Value(quad_vars[cat]) * weights[cat] for cat in categories)
        return result, total_weighted_quad_loss
    else:
        return None, None

# Example usage:
if __name__ == "__main__":
    current_allocations = {
        'transport': 200,
        'food': 1000,
        'housing': 1200,
        'insurance': 50,
        'other_needs': 50,
        'investment': 300,
        'savings': 1000,
    }
    income = 4000
    solution, loss = optimize_budget_with_weighted_quadratic_loss_custom_units(current_allocations, income, scale=1000)
    if solution:
        print("Optimized Effective Allocation (in dollars):")
        for k, v in solution.items():
            print(f"  {k}: {v}")
        print("Total Weighted Quadratic Loss:", loss)
    else:
        print("No feasible solution found.")

Optimized Effective Allocation (in dollars):
  transport: 150
  food: 750
  housing: 1000
  insurance: 50
  other_needs: 50
  investment: 300
  savings: 1350
  total_needs: 2000
  total_wants: 650
Total Weighted Quadratic Loss: 532500
